## 📘 혼잡도 데이터 컬럼 설명

혼잡도 분석에 사용된 주요 변수는 다음과 같습니다.

| 변수명 (`column`) | 변수 설명         | 세부 내용                                | 단위   |
|------------------|------------------|-----------------------------------------|--------|
| `tm`             | 시간              | 날짜 및 시각 (지하철 운행시간 동안만 제공) | -      |
| `line`           | 호선              | 지하철 호선 번호                          | -      |
| `station_number` | 역번호            | 지하철 역 번호                            | -      |
| `station_name`   | 역명              | 지하철 역 이름                            | -      |
| `direction`      | 상하구분          | 지하철 상행 / 하행 구분                   | -      |
| `stn`            | AWS 지점 코드     | 기상청 AWS 관측 지점 코드                  | -      |
| `ta`             | 기온              | 정시 기온                                  | °C     |
| `wd`             | 풍향              | 정시 10분 평균 풍향                        | degree |
| `ws`             | 풍속              | 정시 10분 평균 풍속                        | m/s    |

---


In [4]:
import pandas as pd
import glob
import os

# 연도 리스트
years = ['2021', '2022', '2023']

# 연도별로 병합해서 저장
for year in years:
    # 해당 연도 파일 찾기
    file_paths = sorted(glob.glob(f'data/raw/혼잡도/subway{year}-*.csv'))

    # 파일 병합
    df_list = [pd.read_csv(fp) for fp in file_paths]
    merged_df = pd.concat(df_list, ignore_index=True)

    # 저장 폴더가 없다면 생성
    os.makedirs('data/processed/혼잡도', exist_ok=True)

    # 연도별 파일 저장
    merged_df.to_csv(f'data/processed/혼잡도/subway_{year}.csv', index=False, encoding='utf-8-sig')


    print(f"✅ {year}년 데이터 병합 및 저장 완료: {len(merged_df):,} rows")

✅ 2021년 데이터 병합 및 저장 완료: 5,452,143 rows
✅ 2022년 데이터 병합 및 저장 완료: 5,396,200 rows
✅ 2023년 데이터 병합 및 저장 완료: 5,520,981 rows


In [17]:
import pandas as pd
import glob
import re

def load_and_clean_subway_data(year):
    file_paths = sorted(glob.glob(f'data/raw/혼잡도/subway{year}-*.csv'))
    df_list = []

    for fp in file_paths:
        try:
            df = pd.read_csv(fp, encoding='utf-8')
            # 점(.) 또는 언더바(_) 프리픽스 제거
            df.columns = [re.sub(r'^train_subway\d{2}[._]', '', col) for col in df.columns]
            df_list.append(df)
        except Exception as e:
            print(f"❌ {fp} 읽기 실패: {e}")

    if df_list:
        merged = pd.concat(df_list, ignore_index=True)
        print(f"✅ {year}년 데이터 병합 완료: {len(merged):,} rows × {len(merged.columns)} columns")
        return merged
    else:
        print(f"❗ {year}년 데이터 없음")
        return pd.DataFrame()

# 다시 연도별 병합 시도
df_2021 = load_and_clean_subway_data('2021')
df_2022 = load_and_clean_subway_data('2022')
df_2023 = load_and_clean_subway_data('2023')


✅ 2021년 데이터 병합 완료: 5,452,143 rows × 19 columns
✅ 2022년 데이터 병합 완료: 5,396,200 rows × 19 columns
✅ 2023년 데이터 병합 완료: 5,520,981 rows × 19 columns


In [25]:
print("✅ 2021년 컬럼 예시:", df_2021.columns.tolist()[:10])
print("✅ 2022년 컬럼 예시:", df_2022.columns.tolist()[:10])
print("✅ 2023년 컬럼 예시:", df_2023.columns.tolist()[:10])

✅ 2021년 컬럼 예시: ['tm', 'line', 'station_number', 'station_name', 'direction', 'stn', 'ta', 'wd', 'ws', 'rn_day']
✅ 2022년 컬럼 예시: ['tm', 'line', 'station_number', 'station_name', 'direction', 'stn', 'ta', 'wd', 'ws', 'rn_day']
✅ 2023년 컬럼 예시: ['tm', 'line', 'station_number', 'station_name', 'direction', 'stn', 'ta', 'wd', 'ws', 'rn_day']


In [19]:
df_2021['tm'].head()

0    2021010100
1    2021010101
2    2021010105
3    2021010106
4    2021010107
Name: tm, dtype: int64

In [ ]:
# 연도별 DataFrame이 이미 준비된 상태여야 합니다 (df_2021, df_2022, df_2023)
subway_all = pd.concat([df_2021, df_2022, df_2023], ignore_index=True)

# 결과 확인
print(f"📊 전체 병합 완료: {len(subway_all):,} rows × {len(subway_all.columns)} columns")
subway_all.head()

📊 전체 병합 완료: 16,369,324 rows × 19 columns


,tm,line,station_number,station_name,direction,stn,ta,wd,ws,rn_day,rn_hr1,hm,si,ta_chi,congestion,year,month,day,hour
0,2021010100,1,150,서울역,상선,419,-9.6,291.1,3.3,0.0,0.0,-99.0,-99.0,-12.6,0,2021,1,1,0
1,2021010101,1,150,서울역,상선,419,-9.7,284.6,2.0,0.0,0.0,-99.0,-99.0,-9.8,0,2021,1,1,1
2,2021010105,1,150,서울역,상선,419,-9.3,124.7,2.4,0.0,0.0,-99.0,-99.0,-10.3,1,2021,1,1,5
3,2021010106,1,150,서울역,상선,419,-9.3,126.2,1.7,0.0,0.0,-99.0,-99.0,-10.1,2,2021,1,1,6
4,2021010107,1,150,서울역,상선,419,-9.1,145.7,1.3,0.0,0.0,-99.0,-99.0,-9.7,3,2021,1,1,7


In [26]:
# tm이 숫자형일 경우 먼저 문자열로 변환
subway_all['tm'] = subway_all['tm'].astype(str)

# datetime 형식으로 변환: YYYYMMDDHH
subway_all['datetime'] = pd.to_datetime(subway_all['tm'], format='%Y%m%d%H', errors='coerce')

In [28]:
subway_copy = subway_all.copy()

In [33]:
# subway_copy에서 -99 값을 모두 NaN으로 변환
subway_copy = subway_copy.replace(-99, pd.NA)

In [31]:
df_2021 = subway_copy[subway_copy['datetime'].dt.year == 2021]
df_2022 = subway_copy[subway_copy['datetime'].dt.year == 2022]
df_2023 = subway_copy[subway_copy['datetime'].dt.year == 2023]

In [34]:
weather_cols = ['ta', 'ws', 'wd', 'rn_day', 'rn_hr1']

In [36]:
group_count = subway_copy.groupby(['datetime', 'station_name']).ngroups
print(f"총 그룹 수: {group_count:,}")

총 그룹 수: 7,235,949


In [37]:
# 1. (빠름) groupby → 각 그룹의 line 개수 세기
line_counts = (
    subway_copy
    .groupby(['datetime', 'station_name'])['line']
    .nunique()
    .reset_index(name='n_lines')
)

# 2. (빠름) 2개 이상인 그룹만 추출
multi_line_keys = line_counts[line_counts['n_lines'] > 1][['datetime', 'station_name']]

# 3. (빠름) 원본과 병합하여 해당 그룹만 추출
multi_line_df = subway_copy.merge(multi_line_keys, on=['datetime', 'station_name'], how='inner')


In [38]:
multi_line_df

,tm,line,station_number,station_name,direction,stn,ta,wd,ws,rn_day,rn_hr1,hm,si,ta_chi,congestion,year,month,day,hour,datetime
0,2021010100,1,150,서울역,상선,419,-9.6,291.1,3.3,0.0,0.0,<NA>,<NA>,-12.6,0,2021,1,1,0,2021-01-01 00:00:00
1,2021010101,1,150,서울역,상선,419,-9.7,284.6,2.0,0.0,0.0,<NA>,<NA>,-9.8,0,2021,1,1,1,2021-01-01 01:00:00
2,2021010105,1,150,서울역,상선,419,-9.3,124.7,2.4,0.0,0.0,<NA>,<NA>,-10.3,1,2021,1,1,5,2021-01-01 05:00:00
3,2021010106,1,150,서울역,상선,419,-9.3,126.2,1.7,0.0,0.0,<NA>,<NA>,-10.1,2,2021,1,1,6,2021-01-01 06:00:00
4,2021010107,1,150,서울역,상선,419,-9.1,145.7,1.3,0.0,0.0,<NA>,<NA>,-9.7,3,2021,1,1,7,2021-01-01 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3538073,2023123119,8,2818,가락시장,하선,400,1.4,0.0,0.0,<NA>,<NA>,92.1,<NA>,1.3,34,2023,12,31,19,2023-12-31 19:00:00
3538074,2023123120,8,2818,가락시장,하선,400,0.8,308.8,0.5,<NA>,<NA>,95.3,<NA>,0.7,35,2023,12,31,20,2023-12-31 20:00:00
3538075,2023123121,8,2818,가락시장,하선,400,0.4,320.6,0.1,<NA>,<NA>,96.2,<NA>,0.2,36,2023,12,31,21,2023-12-31 21:00:00
3538076,2023123122,8,2818,가락시장,하선,400,0.0,117.6,0.1,<NA>,<NA>,97.1,<NA>,0.0,32,2023,12,31,22,2023-12-31 22:00:00


In [40]:
check_cols = ['congestion', 'ta_chi']

In [42]:
# 1. 고유값 개수만 먼저 계산
diff_summary = (
    multi_line_df
    .groupby(['datetime', 'station_name'])[check_cols]
    .nunique(dropna=True)
    .reset_index()
)

# 2. 고유값이 2개 이상인 조합만 필터링
mask = diff_summary[check_cols].gt(1).any(axis=1)
diff_keys = diff_summary.loc[mask, ['datetime', 'station_name']]

# 3. 원본에서 해당 조합만 추출
cong_temp_diff_df = multi_line_df.merge(diff_keys, on=['datetime', 'station_name'], how='inner')

In [43]:
cong_temp_diff_df

,tm,line,station_number,station_name,direction,stn,ta,wd,ws,rn_day,rn_hr1,hm,si,ta_chi,congestion,year,month,day,hour,datetime
0,2021010105,1,150,서울역,상선,419,-9.3,124.7,2.4,0.0,0.0,<NA>,<NA>,-10.3,1,2021,1,1,5,2021-01-01 05:00:00
1,2021010106,1,150,서울역,상선,419,-9.3,126.2,1.7,0.0,0.0,<NA>,<NA>,-10.1,2,2021,1,1,6,2021-01-01 06:00:00
2,2021010107,1,150,서울역,상선,419,-9.1,145.7,1.3,0.0,0.0,<NA>,<NA>,-9.7,3,2021,1,1,7,2021-01-01 07:00:00
3,2021010108,1,150,서울역,상선,419,-8.5,194.2,0.6,0.0,0.0,<NA>,0.0,-9.7,3,2021,1,1,8,2021-01-01 08:00:00
4,2021010109,1,150,서울역,상선,419,-7.0,131.8,0.4,0.0,0.0,<NA>,0.27,-8.6,5,2021,1,1,9,2021-01-01 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341123,2023123119,8,2818,가락시장,하선,400,1.4,0.0,0.0,<NA>,<NA>,92.1,<NA>,1.3,34,2023,12,31,19,2023-12-31 19:00:00
3341124,2023123120,8,2818,가락시장,하선,400,0.8,308.8,0.5,<NA>,<NA>,95.3,<NA>,0.7,35,2023,12,31,20,2023-12-31 20:00:00
3341125,2023123121,8,2818,가락시장,하선,400,0.4,320.6,0.1,<NA>,<NA>,96.2,<NA>,0.2,36,2023,12,31,21,2023-12-31 21:00:00
3341126,2023123122,8,2818,가락시장,하선,400,0.0,117.6,0.1,<NA>,<NA>,97.1,<NA>,0.0,32,2023,12,31,22,2023-12-31 22:00:00


In [44]:
check_cols = ['congestion', 'ta_chi']

# 1. 고유값 개수 요약
diff_summary = (
    multi_line_df
    .groupby(['datetime', 'station_name'])[check_cols]
    .nunique(dropna=True)
    .reset_index()
)

# 2. 차이가 있는 그룹만 추출 (둘 중 하나라도 고유값이 2 이상인 경우)
mask = diff_summary[check_cols].gt(1).any(axis=1)
diff_keys = diff_summary.loc[mask, ['datetime', 'station_name']]

# 3. 원본에서 해당 조합만 필터링
cong_temp_diff_df = multi_line_df.merge(diff_keys, on=['datetime', 'station_name'], how='inner')

In [45]:
print(f"⚠️ 혼잡도 또는 체감온도 차이 있는 그룹: {len(cong_temp_diff_df):,} rows")
cong_temp_diff_df.sort_values(['datetime', 'station_name', 'line']).head(10)

⚠️ 혼잡도 또는 체감온도 차이 있는 그룹: 3,341,128 rows


,tm,line,station_number,station_name,direction,stn,ta,wd,ws,rn_day,rn_hr1,hm,si,ta_chi,congestion,year,month,day,hour,datetime
41346,2021010100,3,340,가락시장,상선,400,-6.5,302.6,1.7,0.0,0.0,57.5,<NA>,-9.7,0,2021,1,1,0,2021-01-01
41963,2021010100,3,340,가락시장,하선,400,-6.5,302.6,1.7,0.0,0.0,57.5,<NA>,-9.7,1,2021,1,1,0,2021-01-01
93666,2021010100,8,2818,가락시장,상선,400,-6.5,302.6,1.7,0.0,0.0,57.5,<NA>,-9.7,0,2021,1,1,0,2021-01-01
94283,2021010100,8,2818,가락시장,하선,400,-6.5,302.6,1.7,0.0,0.0,57.5,<NA>,-9.7,1,2021,1,1,0,2021-01-01
38948,2021010100,3,329,고속터미널,상선,889,-6.8,268.7,1.0,0.0,0.0,59.9,<NA>,-6.8,0,2021,1,1,0,2021-01-01
39558,2021010100,3,329,고속터미널,하선,889,-6.8,268.7,1.0,0.0,0.0,59.9,<NA>,-6.8,0,2021,1,1,0,2021-01-01
86294,2021010100,7,2736,고속터미널,상선,889,-6.8,268.7,1.0,0.0,0.0,59.9,<NA>,-6.8,1,2021,1,1,0,2021-01-01
86904,2021010100,7,2736,고속터미널,하선,889,-6.8,268.7,1.0,0.0,0.0,59.9,<NA>,-6.8,0,2021,1,1,0,2021-01-01
64436,2021010100,5,2545,군자,상선,413,-6.4,314.2,1.4,0.0,0.0,52.6,<NA>,-7.0,1,2021,1,1,0,2021-01-01
65047,2021010100,5,2545,군자,하선,413,-6.4,314.2,1.4,0.0,0.0,52.6,<NA>,-7.0,0,2021,1,1,0,2021-01-01


In [46]:
cong_diff_stats = (
    multi_line_df
    .groupby(['datetime', 'station_name'])['congestion']
    .agg(lambda x: x.max() - x.min())
    .reset_index(name='congestion_diff')
)

In [ ]:
# 예: 5 이상 차이나는 경우
cong_diff_keys = cong_diff_stats[cong_diff_stats['congestion_diff'] >= 5][['datetime', 'station_name']]

In [48]:
cong_diff_df = multi_line_df.merge(cong_diff_keys, on=['datetime', 'station_name'], how='inner')

In [49]:
print(f"⚠️ 혼잡도 차이 5 이상인 그룹 행 수: {len(cong_diff_df):,}")
cong_diff_df[['datetime', 'station_name', 'line', 'congestion']].sort_values(['datetime', 'station_name']).head(10)

⚠️ 혼잡도 차이 5 이상인 그룹 행 수: 3,138,350


,datetime,station_name,line,congestion
36042,2021-01-01 05:00:00,가락시장,3,3
36629,2021-01-01 05:00:00,가락시장,3,0
81248,2021-01-01 05:00:00,가락시장,8,6
81835,2021-01-01 05:00:00,가락시장,8,4
17436,2021-01-01 05:00:00,건대입구,2,3
18018,2021-01-01 05:00:00,건대입구,2,7
73328,2021-01-01 05:00:00,건대입구,7,4
73910,2021-01-01 05:00:00,건대입구,7,12
19730,2021-01-01 05:00:00,교대,2,9
20306,2021-01-01 05:00:00,교대,2,12


In [50]:
subway_copy.isna().sum().sort_values(ascending=False).head(20)

si                6064242
hm                 844594
rn_hr1             360796
rn_day             351574
wd                 230786
ws                 230786
ta                 216468
ta_chi                352
hour                    0
day                     0
month                   0
year                    0
congestion              0
tm                      0
line                    0
stn                     0
direction               0
station_name            0
station_number          0
datetime                0
dtype: int64

### si 변수 확인

In [51]:
# 일사량 값이 존재하는 행만 필터링
sunny_df = subway_copy[subway_copy['si'].notna()]

In [52]:
# 중복 제거한 지상역 리스트
sunny_stations = sunny_df['station_name'].dropna().unique()
print(f"🌞 일사량 값이 존재한 역 개수: {len(sunny_stations)}개")

🌞 일사량 값이 존재한 역 개수: 324개


In [53]:
# 알파벳 순으로 상위 20개만 보기
sorted(sunny_stations)[:20]

['가능',
 '가락시장',
 '가산디지털단지',
 '간석',
 '강남',
 '강남구청',
 '강동',
 '강동구청',
 '강변',
 '강일',
 '개봉',
 '개화산',
 '건대입구',
 '경마공원',
 '경복궁',
 '경찰병원',
 '고덕',
 '고려대',
 '고속터미널',
 '고잔']

In [54]:
subway_copy = subway_copy.drop(columns=['si'])

불필요한 컬럼 삭제 - 일사량(`si`)
- 기상청 ASOS 연계값으로, 역 자체의 구조(지상/지하)와는 무관
- 분석 목적과 부합하지 않으므로 제거

In [55]:
rename_cols = {
    'ta': 'TMP',
    'ta_chi': 'CHT',        # 체감온도 (Custom)
    'rn_hr1': 'PCP',
    'rn_day': 'DAY_PCP',    # 일 누적 강수량 (Custom)
    'ws': 'WSD',
    'wd': 'VEC',
    'hm': 'REH',
}

subway_copy = subway_copy.rename(columns=rename_cols)

컬럼명 기상청 표준 변수명으로 변경
- `ta` → `TMP` (기온)
- `rn_hr1` → `PCP` (1시간 강수량)
- `ws` → `WSD` (풍속)
- `wd` → `VEC` (풍향)
- `hm` → `REH` (상대습도)
- congestion : 혼잡도 그대로

In [56]:
subway_copy

,tm,line,station_number,station_name,direction,stn,TMP,VEC,WSD,DAY_PCP,PCP,REH,CHT,congestion,year,month,day,hour,datetime
0,2021010100,1,150,서울역,상선,419,-9.6,291.1,3.3,0.0,0.0,<NA>,-12.6,0,2021,1,1,0,2021-01-01 00:00:00
1,2021010101,1,150,서울역,상선,419,-9.7,284.6,2.0,0.0,0.0,<NA>,-9.8,0,2021,1,1,1,2021-01-01 01:00:00
2,2021010105,1,150,서울역,상선,419,-9.3,124.7,2.4,0.0,0.0,<NA>,-10.3,1,2021,1,1,5,2021-01-01 05:00:00
3,2021010106,1,150,서울역,상선,419,-9.3,126.2,1.7,0.0,0.0,<NA>,-10.1,2,2021,1,1,6,2021-01-01 06:00:00
4,2021010107,1,150,서울역,상선,419,-9.1,145.7,1.3,0.0,0.0,<NA>,-9.7,3,2021,1,1,7,2021-01-01 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16369319,2023123119,8,2828,남위례,하선,572,0.6,0.0,0.0,7.0,0.0,83.1,0.0,18,2023,12,31,19,2023-12-31 19:00:00
16369320,2023123120,8,2828,남위례,하선,572,0.0,354.7,0.0,7.0,0.0,84.7,-0.6,17,2023,12,31,20,2023-12-31 20:00:00
16369321,2023123121,8,2828,남위례,하선,572,-0.6,0.0,0.0,7.0,0.0,85.1,-1.1,21,2023,12,31,21,2023-12-31 21:00:00
16369322,2023123122,8,2828,남위례,하선,572,-0.8,0.0,0.0,7.0,0.0,85.6,-1.3,18,2023,12,31,22,2023-12-31 22:00:00


In [57]:
subway_copy.isna().sum().sort_values(ascending=False).head(20)

REH               844594
PCP               360796
DAY_PCP           351574
VEC               230786
WSD               230786
TMP               216468
CHT                  352
hour                   0
day                    0
month                  0
year                   0
congestion             0
tm                     0
line                   0
stn                    0
direction              0
station_name           0
station_number         0
datetime               0
dtype: int64

풍향(`VEC`) 변수 삭제
- 단위: 도(°)
- 분석 목적상 직접적인 의미를 파악하거나 모델에 활용하기 어려움
- 기상 요소 중에서도 해석력이 낮은 편이므로 제거


In [69]:
del subway_all

In [71]:
import os
os.makedirs("data/tmp_split", exist_ok=True)

In [72]:
chunk_size = 100_000  # 10만 행씩 저장
total_rows = len(subway_copy)

for i in range(0, total_rows, chunk_size):
    chunk = subway_copy.iloc[i:i+chunk_size]
    chunk.to_csv(f"data/tmp_split/subway_copy_part_{i//chunk_size + 1}.csv", index=False)
    print(f"✅ 저장 완료: subway_copy_part_{i//chunk_size + 1}.csv")

✅ 저장 완료: subway_copy_part_1.csv
✅ 저장 완료: subway_copy_part_2.csv
✅ 저장 완료: subway_copy_part_3.csv
✅ 저장 완료: subway_copy_part_4.csv
✅ 저장 완료: subway_copy_part_5.csv
✅ 저장 완료: subway_copy_part_6.csv
✅ 저장 완료: subway_copy_part_7.csv
✅ 저장 완료: subway_copy_part_8.csv
✅ 저장 완료: subway_copy_part_9.csv
✅ 저장 완료: subway_copy_part_10.csv
✅ 저장 완료: subway_copy_part_11.csv
✅ 저장 완료: subway_copy_part_12.csv
✅ 저장 완료: subway_copy_part_13.csv
✅ 저장 완료: subway_copy_part_14.csv
✅ 저장 완료: subway_copy_part_15.csv
✅ 저장 완료: subway_copy_part_16.csv
✅ 저장 완료: subway_copy_part_17.csv
✅ 저장 완료: subway_copy_part_18.csv
✅ 저장 완료: subway_copy_part_19.csv
✅ 저장 완료: subway_copy_part_20.csv
✅ 저장 완료: subway_copy_part_21.csv
✅ 저장 완료: subway_copy_part_22.csv
✅ 저장 완료: subway_copy_part_23.csv
✅ 저장 완료: subway_copy_part_24.csv
✅ 저장 완료: subway_copy_part_25.csv
✅ 저장 완료: subway_copy_part_26.csv
✅ 저장 완료: subway_copy_part_27.csv
✅ 저장 완료: subway_copy_part_28.csv
✅ 저장 완료: subway_copy_part_29.csv
✅ 저장 완료: subway_copy_part_30.csv
✅ 저장 완료: subway_cop